## CNN

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### nn.Conv2d : CONV Layer

- 원하는 이미지 사이즈에 따라 kernel size, stride, padding을 설정할 수 있음
- 일반적으로 kernel=3, stride=1, padding=1을 하면 원래 이미지 사이즈가 유지되며 max pool등을 통해 이미지 사이즈를 반으로 줄임

In [3]:
x = torch.randn(128, 1, 28, 28).to(device)

* filter : 1, out_channels : 32, stride=1

In [4]:
conv_layer = nn.Conv2d(1, 32, kernel_size=3) # filter : 1, out_channels : 32
x_after_conv = conv_layer(x)
print('x_after_conv:',x_after_conv.shape)

x_after_conv: torch.Size([128, 32, 26, 26])


* stride=2

In [5]:
conv_layer = nn.Conv2d(1, 32, kernel_size=3, stride=2)
x_after_conv = conv_layer(x)
print('x_after_conv:',x_after_conv.shape)

x_after_conv: torch.Size([128, 32, 13, 13])


In [6]:
conv_layer = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
x_after_conv = conv_layer(x)
print('x_after_conv:',x_after_conv.shape)

x_after_conv: torch.Size([128, 32, 28, 28])


### Max Pooling

In [7]:
x = torch.randn(128, 1, 28, 28).to(device)

In [8]:
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
x_after_maxpool = maxpool(x)
print('x_after_maxpool:',x_after_maxpool.shape)

x_after_maxpool: torch.Size([128, 1, 14, 14])


In [9]:
maxpool = nn.MaxPool2d(kernel_size=4, stride=4)
x_after_maxpool = maxpool(x)
print('x_after_maxpool:',x_after_maxpool.shape)

x_after_maxpool: torch.Size([128, 1, 7, 7])


### Average Pooling

In [10]:
x = torch.randn(128, 1, 28, 28).to(device)

In [11]:
avgpool = nn.AvgPool2d(kernel_size=2, stride=2)
x_after_avgpool = avgpool(x)
print('x_after_avgpool:',x_after_avgpool.shape)

x_after_avgpool: torch.Size([128, 1, 14, 14])


In [12]:
avgpool = nn.AvgPool2d(kernel_size=7, stride=7)
x_after_avgpool = avgpool(x)
print('x_after_avgpool:',x_after_avgpool.shape)

x_after_avgpool: torch.Size([128, 1, 4, 4])


### Global Average Pooling

In [13]:
x = torch.randn(128, 1, 28, 28).to(device)

In [14]:
global_avg_pool = nn.AdaptiveAvgPool2d((1,1)) # 이미지 출력 값을 (1,1)로 
x_after_global_avg_pool = global_avg_pool(x)
print('x_after_global_avg_pool:',x_after_global_avg_pool.shape)

x_after_global_avg_pool: torch.Size([128, 1, 1, 1])


# CNN application

In [15]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# MNIST 데이터셋 
train_data = datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

# Data loader
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [16]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) # input=1, output=32
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # input은 이전의 output channel이므로 32
        self.relu = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.mlp = nn.Linear(64*7*7, num_classes)  # print('out 2:',out.shape) # [128, 64, 7, 7]
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.maxpool1(out)
        # print('out 1:',out.sha pe) # [128, 32, 14, 14]
        
        out = self.conv2(out)
        out = self.relu(out)
        out = self.maxpool2(out)
        # print('out 2:',out.shape) # [128, 64, 7, 7]
        
        out = out.view(out.size(0), -1)
        out = self.mlp(out) # 마지막 classification을 위해 필요
        
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_Model(num_classes=10).to(device)

In [17]:
CELoss = nn.CrossEntropyLoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 최적화 알고리즘 class 선언

* 모델 학습

In [18]:
# 뉴럴 네트워크 모델 학습
total_epochs = 3
print('number of iteration :', len(train_loader))
# epoch : 모든 데이터를 한 번 학습하는 단위
for epoch in range(total_epochs):
    # iteration : 한 'mini-batch' 단위의 데이터를 학습하는 단위
    for i, (images, labels) in enumerate(train_loader):  
        # images : [mini-batch, 1, 28, 28]
        # labels : [mini-batch]
#         images = images.reshape(-1, 28*28).to(device) 
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        ce_loss = CELoss(outputs, labels)
        
        # Backward and optimize
        adam_optimizer.zero_grad() # 다양한 optimization 기법 적용 가능
        ce_loss.backward() # Back propagation
        adam_optimizer.step() # optimizer 작동
            
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epochs, ce_loss.item()))

number of iteration : 469
Epoch [1/3], Loss: 0.0309
Epoch [2/3], Loss: 0.0347
Epoch [3/3], Loss: 0.0217


* 모델 성능 테스트

In [19]:
# 학습이 끝난 후 모델 성능 테스트
# test에서는 back propagation 작업을 하지 않으므로 gradient를 계산하지 않도록 함 - 메모리의 효율성을 위해

model.eval()
with torch.no_grad(): # gradient 계산하지 않도록 하는 코드
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.48 %


In [20]:
# 학습한 모델을 model_CNN.ckpt라는 이름으로 저장
torch.save(model.state_dict(), 'model_CNN.ckpt')

* 학습한 모델 다시 불러오기(parameter)

In [22]:
model.load_state_dict(torch.load('model_CNN.ckpt'))

<All keys matched successfully>